# Import necessary modules and libraries

In [1]:
import PyQt5
%config InlineBackend.figure_format = 'retina'
# %matplotlib inline
%matplotlib qt5
#python packages
import numpy as np
# import cupy as cp
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import pac
import warnings

import seaborn as sns
sns.set_style('white')

import imp
import shape
import utils
from eegstats import eegstats
from load_features import load_WaveformShape_features, Bandpower_features, mean_and_peak_freqs, statistics

imp.reload(utils)
imp.reload(shape)

<module 'shape' from 'C:\\Users\\USER\\Documents\\Yachay_Tech\\Thesis_Project\\ParkinsonsDetection\\python_scripts\\shape.py'>

# Load data and meta-data

In [2]:
## Load data and meta-data
"""Which 'comparison' to make:
    1. Off-med vs Controls
    2. On-med vs Controls
    3. Off-med vs On-med
    """
comparison = 1
data_repository = 'UNM' #USDiego for dataset from University of San Diego, UNM for dataset from U New Mexico
all_chan = False
EO = False
bands = [[0.5,4], [4,8], [8,12], [16,32], [32,64]] #Delta, Theta, Alpha, Beta, Gamma

In [3]:
Fs, t, S, Sc, Smed, flo, fhi = utils.loadmeta() 
eeg,rejects = utils.loadPD(EO, all_chan, data_repository) # EO means Eyes Opened

In [4]:
# for i,j,k in zip(range(0,len(eeg['off'])), range(0,len(eeg['on'])), range(0,len(eeg['C']))):
#     eeg['off'][:][i] = (eeg['off'][:][i]-np.mean(eeg['off'][:][i]))/np.max(abs(eeg['off'][:][i]))
#     eeg['on'][:][j] = (eeg['on'][:][j]-np.mean(eeg['on'][:][j]))/np.max(abs(eeg['on'][:][j]))
#     eeg['C'][:][k] = (eeg['C'][:][k]-np.mean(eeg['C'][:][k]))/np.max(abs(eeg['C'][:][k]))

# Ploting a signal

In [5]:
# Plot a signal
sns.set(font_scale=1.2)
data = eeg['off'][1]
time = np.arange(data.size) / Fs

fig, ax = plt.subplots(1, 1, figsize=(12, 4))
plt.plot(time, data, lw=1.5, color='k')
plt.xlabel('Time (seconds)')
plt.ylabel('Voltage')
plt.xlim([time.min(), time.max()])
plt.title('EEG Signal for PD patients off-medication')
sns.despine()

# plt.plot(eeg['off'][6,0:10],label='control')
# plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))

# CNN

# Load dataset with all features

In [51]:
# create features of class I
cl_B    = np.ones((S,1)) # 1

In [52]:
# create features of class II
if comparison == 1 or comparison == 3:
    cl_C    = np.zeros((Smed,1)) # transition means 0 #Original line
elif comparison == 2:
    cl_C    = np.ones((Smed,1))

In [53]:
# create features of class III
cl_E    = np.zeros((Sc,1)) # -1

In [54]:
# X_off = np.zeros((28165,1526))
# for i in range(len(eeg['off'])):
#     X_off[:,i] = eeg['off'][i][:28165]
    
# X_C = np.zeros((28165,1509))
# for i in range(len(eeg['C'])):
#     X_C[:,i] = eeg['C'][i][:28165]

# X = np.concatenate((X_off, X_C), axis=1)

In [55]:
len_signal = 2347 #28165 max

X_off = np.zeros((S,len_signal))
for i in range(len(eeg['off'])):
    X_off[i,:] = eeg['off'][i][:len_signal]
    
X_C = np.zeros((Sc,len_signal))
for i in range(len(eeg['C'])):
    X_C[i,:] = eeg['C'][i][:len_signal]

X = np.concatenate((X_off, X_C), axis=0)
# X = normalize(X)

In [56]:
y = np.concatenate((np.ravel(cl_B), np.ravel(cl_E)), axis=0)

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=425156)

In [58]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
# y_train

In [59]:
from keras.utils import to_categorical
y_test = to_categorical(y_test)
# y_test

In [60]:
import pandas as pd
import keras.backend as K
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical 
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import Model
import timeit
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution1D, MaxPooling1D, ZeroPadding1D
from keras.optimizers import SGD
#import cv2, numpy as np
import warnings
warnings.filterwarnings('ignore')

In [61]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [62]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1], 1)

In [63]:
X_train.shape

(453, 2347, 1)

In [64]:
batch_size = 256
num_classes = 2
epochs = 30
input_shape=(X_train.shape[1], 1)
# input_shape =(X_train.shape[0],X_train.shape[1], 1)

In [65]:
print(input_shape)

(2347, 1)


In [66]:
from keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense
from keras.regularizers import l2

In [67]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


# LSTM and CNN

In [100]:
model = Sequential()
# model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(256, kernel_size=3,padding = 'same',activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(2, activation='sigmoid'))
model.summary()
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=3, batch_size=64)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_20 (Conv1D)           (None, 2347, 256)         1024      
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 1173, 256)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1173, 256)         0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 1173, 128)         98432     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 586, 128)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 586, 128)          0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 100)             

In [101]:
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [102]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

Epoch 1/30
2/2 [==============================] - 1s 313ms/step - loss: 0.6860 - accuracy: 0.5077
Epoch 2/30
2/2 [==============================] - 0s 220ms/step - loss: 0.6751 - accuracy: 0.5077
Epoch 3/30
2/2 [==============================] - 0s 220ms/step - loss: 0.6687 - accuracy: 0.5121
Epoch 4/30
2/2 [==============================] - 0s 216ms/step - loss: 0.6634 - accuracy: 0.5121
Epoch 5/30
2/2 [==============================] - 0s 222ms/step - loss: 0.6582 - accuracy: 0.5276
Epoch 6/30
2/2 [==============================] - 0s 222ms/step - loss: 0.6494 - accuracy: 0.5497
Epoch 7/30
2/2 [==============================] - 0s 221ms/step - loss: 0.6453 - accuracy: 0.5894
Epoch 8/30
2/2 [==============================] - 0s 224ms/step - loss: 0.6303 - accuracy: 0.6578
Epoch 9/30
2/2 [==============================] - 0s 219ms/step - loss: 0.6174 - accuracy: 0.6711
Epoch 10/30
2/2 [==============================] - 0s 222ms/step - loss: 0.6121 - accuracy: 0.6865
Epoch 11/30
2/2 [==

In [103]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 0s 20ms/step - loss: 0.6223 - accuracy: 0.6923
Test loss: 0.6223288178443909
Test accuracy: 0.692307710647583


# Convolutional Only

In [104]:
model = Sequential()
# intput_shape=(x_train.shape[1], 1)
model.add(Conv1D(128, kernel_size=3,padding = 'same',activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(128,kernel_size=3,padding = 'same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))
#model.add(Conv1D(64,kernel_size=3,padding = 'same', activation='relu'))
#model.add(MaxPooling1D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_22 (Conv1D)           (None, 2347, 128)         512       
_________________________________________________________________
batch_normalization_3 (Batch (None, 2347, 128)         512       
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 1173, 128)         0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 1173, 128)         49280     
_________________________________________________________________
batch_normalization_4 (Batch (None, 1173, 128)         512       
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 586, 128)          0         
_________________________________________________________________
flatten (Flatten)            (None, 75008)           

In [105]:
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [106]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

Epoch 1/30
2/2 [==============================] - 0s 208ms/step - loss: 0.7407 - accuracy: 0.5607
Epoch 2/30
2/2 [==============================] - 0s 163ms/step - loss: 0.6174 - accuracy: 0.6976
Epoch 3/30
2/2 [==============================] - 0s 161ms/step - loss: 0.6061 - accuracy: 0.7042
Epoch 4/30
2/2 [==============================] - 0s 160ms/step - loss: 0.5892 - accuracy: 0.7241
Epoch 5/30
2/2 [==============================] - 0s 160ms/step - loss: 0.5754 - accuracy: 0.7263
Epoch 6/30
2/2 [==============================] - 0s 161ms/step - loss: 0.5326 - accuracy: 0.7594
Epoch 7/30
2/2 [==============================] - 0s 163ms/step - loss: 0.5277 - accuracy: 0.7550
Epoch 8/30
2/2 [==============================] - 0s 161ms/step - loss: 0.5180 - accuracy: 0.7748
Epoch 9/30
2/2 [==============================] - 0s 161ms/step - loss: 0.5014 - accuracy: 0.7815
Epoch 10/30
2/2 [==============================] - 0s 162ms/step - loss: 0.4975 - accuracy: 0.7881
Epoch 11/30
2/2 [==

In [107]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 0s 10ms/step - loss: 1.2678 - accuracy: 0.5333
Test loss: 1.2678425312042236
Test accuracy: 0.5333333611488342
